In [7]:
# Extract AMC Data from numina

from datasets import load_dataset

ds = load_dataset("AI-MO/NuminaMath-CoT")
# Filter for amc_aime problems
olympiad = ds['train'].filter(lambda x: x['source'] == 'olympiads')

In [8]:
from pprint import pprint
from rllm.grading.grader import extract_boxed_answer
idx = 14081
#pprint(olympiad[idx])
pprint(olympiad[idx])
print('### Conclusion' in olympiad[idx]['solution'])
extract_boxed_answer(olympiad[idx]['solution'])

{'messages': [{'content': 'Prove that the sum of the lengths of the diagonals '
                          'of a convex pentagon \\(ABCDE\\) is greater than '
                          'the perimeter but less than twice the perimeter.',
               'role': 'user'},
              {'content': 'Consider a convex pentagon $ABCDE$. We need to show '
                          'that the sum of the lengths of its diagonals is '
                          'greater than its perimeter but less than twice its '
                          'perimeter.\n'
                          '\n'
                          '1. Denote the lengths of the sides of the pentagon '
                          'as follows:\n'
                          '    \\[\n'
                          '    AB = a, \\, BC = b, \\, CD = c, \\, DE = d, \\, '
                          'EA = e\n'
                          '    \\]\n'
                          '\n'
                          '2. Denote the lengths of the diagonals as follow

In [9]:
PROOF_PROMPT = """Your task is to identify if the user provided problem into two categories:
Case 1: Problems that require a proof.
Case 2: Problems that have a clear and direct answer. If a problem asks for a proof and has a direct answer, it still falls under this case.
Case 3: It's not a math problem and it just making a blanket statement. 

Output 1 if it falls under Case 1. Output 2 if it falls under Case 2. Output 3 if it falls under Case 3. Only output 1 or 2 or 3 (at most one token!).

You are provided several examples of Case 1 and 2 below:

Case 1:

Prove that if \( \frac{a}{b} = \frac{b}{c} \), then \( a^{2} + c^{2} \geq 2 b^{2} \).

Let \(a, b,\) and \(c\) be strictly positive real numbers such that \(abc = 1\). Show that

$$
\left(a+\frac{1}{b}\right)^{2}+\left(b+\frac{1}{c}\right)^{2}+\left(c+\frac{1}{a}\right)^{2} \geq 3(a+b+c+1)
$$

Prove that the sum of the lengths of the diagonals of a convex '
            'pentagon \\(ABCDE\\) is greater than the perimeter but less than '
            'twice the perimeter.

Case 2:

Find all prime numbers \( p \) such that for any prime number \( q < p \), if \( p = kq + r \) with \( 0 \leq r < q \), then there does not exist an integer \( a > 1 \) such that \( a^2 \) divides \( r \).

Determine the value of
$$
z=a \sqrt{a} \sqrt[4]{a} \sqrt[8]{a} \ldots \sqrt[2^{n}]{a} \ldots
$$
if \( n \) is infinitely large.

A set consists of five different odd positive integers, each greater than 2. When these five integers are multiplied together, their product is a five-digit integer of the form $AB0AB$, where $A$ and $B$ are digits with $A \neq 0$ and $A \neq B$. (The hundreds digit of the product is zero.) In total, how many different sets of five different odd positive integers have these properties?


Find all integers \(a\) such that the equation
$$
x^{2} + axy + y^{2} = 1
$$
has infinitely many integer solutions \((x, y)\). Prove your conclusion.

Suppose a hyperbola \( C: \frac{x^{2}}{a^{2}} - \frac{y^{2}}{b^{2}} = 1 \) has a right focal point \( F \). Let \( P \) be a point outside the hyperbola. Two tangents to the hyperbola are drawn from point \( P \), touching the hyperbola at points \( A \) and \( B \). If \( A B \perp P F \), find the locus of point \( P \).

The user provides both the problem and solution below. Use this information to make your best informed decision.
"""

In [10]:
import vertexai
from vertexai.generative_models import GenerativeModel
import re
import ast
from google.cloud.aiplatform_v1beta1.types.content import SafetySetting
from vertexai.generative_models import GenerativeModel, HarmBlockThreshold, HarmCategory

HARM_CATEGORIES = [
    HarmCategory.HARM_CATEGORY_UNSPECIFIED,
    HarmCategory.HARM_CATEGORY_HARASSMENT,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH,
]


def call_gemini_llm(
    prompt: str,
    system_prompt: str,
    project_id: str = 'cloud-llm-test',
    location: str = "us-central1",
    model_id: str = "gemini-1.5-pro-002",
) -> str:
    """
    Calls a Gemini LLM on Vertex AI to generate a response.
    
    Args:
        project_id (str): Your GCP project ID.
        location (str): The region to use (e.g., us-central1).
        model_id (str): The specific Gemini model resource name.
        prompt (str): The text prompt to send to the LLM.
    
    Returns:
        str: The generated text from the Gemini model.
    """

    # Initialize the Vertex AI environment
    vertexai.init(project=project_id, location=location)

    model = GenerativeModel(
        model_name=model_id,
        system_instruction=[
            system_prompt
        ],
    )
    try:
        response = model.generate_content([prompt], safety_settings=[
                SafetySetting(category=h, threshold=HarmBlockThreshold.BLOCK_NONE)
                for h in HARM_CATEGORIES
            ])
    except Exception as e:
        print(e)
        return None
    # text is not part of response
    try:
        response.text
        return response.text
    except:
        return None

In [11]:
import concurrent.futures
from rllm.data.numina.utils import extract_answer, get_multiple_choice_answer

def process_entry(entry):
    output_dict = {}
    # 1) Get the problem text
    problem_text = entry['problem']
    solution_text = entry['solution']
    # 2) Call Gemini LLM
    output_str = call_gemini_llm(f'Problem: {problem_text} \n\n Solution: {solution_text}', system_prompt=PROOF_PROMPT)
    if not output_str:
        print("Gemini not happy.")
        return None
    if '2' in output_str:
        output_dict['problem'] = entry['problem']
        output_dict['solution'] = entry['solution']
        return {
            'problem': entry['problem'],
            'solution': entry['solution'],
        }
    else:
        if '3' in output_str:
            print(output_str, entry['problem'])
    return output_dict


from tqdm import tqdm
import json
subset = olympiad
final_list = []
counter = 0
# Use a ProcessPoolExecutor with tqdm to display progress
with concurrent.futures.ProcessPoolExecutor(max_workers=48) as executor:
    # Wrap executor.map with tqdm to track progress of parallel tasks
    results = list(tqdm(executor.map(process_entry, subset), total=len(subset), desc="Processing entries"))

results = [r for r in results if r]

# Save final list as json
with open("olympiad_processed.json", "w") as f:
    json.dump(results, f, indent=2)

3
 What amazing conclusion can be drawn from the answer: "There is something instead of nothing"?
3
 One tour guide once complained to a small group of tourists: “When I have a tour group of 50 people, after giving them free time during a long excursion, the tourists almost always arrive on time at the appointed time. But if there are few people in the tour group, I constantly encounter the problem that several people do not show up at the designated time.” 

Explain to the tour guide the reason for such behavior by providing at least two different economic arguments.
3
 Free the stick. We have a loop of rope threaded through one end of a stick, as shown in the picture, but too short to be looped over the opposite end. The puzzle is to suspend the stick from the buttonhole of a jacket (see the picture) and then release it again.
3

 Three lines, intersecting at one point, lie on the same line.3

 What does the expression "necessary and sufficient condition" mean? Explain the meaning of

Processing entries:   4%|▍         | 6515/150563 [00:00<00:02, 65129.25it/s]

3
 Use this definition for a more concise formulation of some of the previous questions.
3
 Each player plays against every other player exactly once.
3
 $$\tan (n+1)^{\circ}-\tan n^{\circ} =\frac{\sin (n+1)^{\circ}}{\cos (n+1)^{\circ}}-\frac{\sin n^{\circ}}{\cos n^{\circ}}=\frac{\sin (n+1)^{\circ} \cos n^{\circ}-\cos (n+1)^{\circ} \sin n^{\circ}}{\cos n^{\circ} \cdot \cos (n+1)^{\circ}} =\frac{\sin 1^{\circ}}{\cos n^{\circ} \cdot \cos (n+1)^{\circ}}.$$
3
 Write down an integer \( N \) between 0 and 10, inclusive. You will receive \( N \) points unless another team writes down the same \( N \), in which case you will receive nothing.
3
 It is considered one of the "Three Major Scientific Plans" along with the Manhattan Project and the Apollo Program. This plan was officially launched in 1990 and involved scientists from the United States, the United Kingdom, France, Germany, Japan, and China.

A. New Energy Regeneration Plan
B. Space Migration Plan
C. Human Genome Project
D. Solar Powe

Processing entries:   5%|▌         | 8236/150563 [00:12<04:55, 482.35it/s]  

3
 Consider a person walking towards the North Pole. The markings on the compass, as everyone knows, look like

\[
\mathrm{w}_{\mathrm{S}}^{\mathrm{N}} \mathrm{E}
\]

He reaches the pole and, having passed through it, has to turn around to look north. Now, east is on his left, and west is on his right, so the markings on the compass appear as

\[
E{ }_{S}^{N} w
\]

which seems absurd.

How can this paradox be explained?

We stand with a child in front of a large mirror that reflects us fully.
- Why, asks the clever youngster, when I look in the mirror, do my right and left sides switch places, but my top and bottom do not? If the mirror changes sides in the horizontal direction, why doesn't it do so in the vertical direction? Why don't I see myself standing on my head?
3
 Provide at least two different arguments that could justify the interest of the Central Bank of the Russian Federation in using the described loyalty program. Justify your assumptions.

Provide at least two different 

...

3
 If we write the progression $\div 4, 8, (16, 32), \ldots, \left(2^{2n}, 2^{2n+1}\right)$, then every pair of numbers in brackets gives a perfect number if we multiply the larger number in the pair, decreased by one, by the smaller number.

Note: Provided that $2^{2n+1} - 1$ is a prime number.


Processing entries: 100%|██████████| 150563/150563 [22:16<00:00, 112.66it/s]


I start?


Building final list:  31%|███       | 46005/150563 [01:07<02:33, 681.14it/s] 


KeyboardInterrupt: 

In [16]:

results = [r for r in results if r]
len(results)

102829

In [17]:
with open("olympiad_processed.json", "w") as f:
    json.dump(results, f, indent=2)